# Capstone Project Battle of the Neighborhoods - Houston

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

! pip install folium==0.5.0
import folium # plotting library

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


print('Libraries imported.')

     |████████████████████████████████| 79 kB 7.4 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=1e23bc136a5270d03218a5ef527c65ddb9ce95dcf9ec14b681874d12c998052b
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium
Libraries imported.


In [2]:
!wget -q -O 'h_data.csv' https://docs.google.com/spreadsheets/d/1HviOJilytLsUzM27PtmuYx3I6lQDKcPMx9MfioRkk9k/gviz/tq?tqx=out:csv
print('Data downloaded!')

Data downloaded!


In [3]:
neighborhoods = pd.read_csv("h_data.csv")


In [4]:
neighborhoods

,Neighborhood,Latitude,Longitude
0,Garden Oaks / Oak Forest,29.826345,-95.417523
1,The Heights / Timbergrove,29.796311,-95.413312
2,Memorial,29.770561,-95.508688
3,Uptown / Galleria,29.740464,-95.479825
4,Memorial Park / Rice Military,29.770999,-95.406581
5,River Oaks,29.750225,-95.426855
6,Highland Village,29.736313,-95.448954
7,Shepherd Kirby,29.736508,-95.413534
8,Montrose,29.752623,-95.384910
9,Downtown / Midtown,29.756378,-95.363284


In [14]:
#Map of Target Neighborhoods in Houston, TX

address = 'Houston, TX'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Manhattan using latitude and longitude values
map_houston = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_houston)  
    
map_houston

In [6]:
#Define 4Square Credentials
CLIENT_ID = 'Y3CRNKK4MYSH0MUSM05PCWBBXJQ4TBBHXIGSFJSC15OJS13U' # your Foursquare ID
CLIENT_SECRET = 'QEGJVIW1HUVLTHVWRVF4AO5FVTXFP5VSFLNEZ2I5MSZ32Y0A' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 200 # A default Foursquare API limit value
SECTION= 'food'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Y3CRNKK4MYSH0MUSM05PCWBBXJQ4TBBHXIGSFJSC15OJS13U
CLIENT_SECRET:QEGJVIW1HUVLTHVWRVF4AO5FVTXFP5VSFLNEZ2I5MSZ32Y0A


In [7]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
#Function to get top venues from each Target Neighborhood in Houston
#Set radius at 2,000m as average neighborhood has a diameter of about 4,000 meters
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&section={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            SECTION,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',          
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
houston_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Garden Oaks / Oak Forest
The Heights / Timbergrove
Memorial
Uptown / Galleria
Memorial Park / Rice Military
River Oaks
Highland Village
Shepherd Kirby
Montrose
Downtown / Midtown
Museum District
West University
Medical Center
Bellaire


In [10]:
houston_venues.shape

(1235, 7)

In [12]:
# one hot encoding
houston_onehot = pd.get_dummies(houston_venues[['Venue Category']], prefix="", prefix_sep="")
houston_onehot.head()

,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,Breakfast Spot,Burger Joint,Burrito Place,Cafeteria,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Churrascaria,Comfort Food Restaurant,Creperie,Cuban Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Empanada Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fondue Restaurant,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Greek Restaurant,Halal Restaurant,Hawaiian Restaurant,Hot Dog Joint,Hotpot Restaurant,Hunan Restaurant,Indian Chinese Restaurant,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Latin American Restaurant,Mac & Cheese Joint,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Mongolian Restaurant,New American Restaurant,Noodle House,North Indian Restaurant,Peruvian Restaurant,Pizza Place,Poke Place,Portuguese Restaurant,Ramen Restaurant,Restaurant,Salad Place,Salvadoran Restaurant,Sandwich Place,Seafood Restaurant,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
# add neighborhood column back to dataframe
houston_onehot['Neighborhood'] = houston_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [houston_onehot.columns[-1]] + list(houston_onehot.columns[:-1])
houston_onehot = houston_onehot[fixed_columns]

houston_onehot.head()

,Neighborhood,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,Breakfast Spot,Burger Joint,Burrito Place,Cafeteria,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Churrascaria,Comfort Food Restaurant,Creperie,Cuban Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Empanada Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fondue Restaurant,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Greek Restaurant,Halal Restaurant,Hawaiian Restaurant,Hot Dog Joint,Hotpot Restaurant,Hunan Restaurant,Indian Chinese Restaurant,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Latin American Restaurant,Mac & Cheese Joint,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Mongolian Restaurant,New American Restaurant,Noodle House,North Indian Restaurant,Peruvian Restaurant,Pizza Place,Poke Place,Portuguese Restaurant,Ramen Restaurant,Restaurant,Salad Place,Salvadoran Restaurant,Sandwich Place,Seafood Restaurant,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Garden Oaks / Oak Forest,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Garden Oaks / Oak Forest,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Garden Oaks / Oak Forest,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Garden Oaks / Oak Forest,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Garden Oaks / Oak Forest,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
houston_grouped = houston_onehot.groupby('Neighborhood').mean().reset_index()
houston_grouped

,Neighborhood,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,Breakfast Spot,Burger Joint,Burrito Place,Cafeteria,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Churrascaria,Comfort Food Restaurant,Creperie,Cuban Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Empanada Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fondue Restaurant,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Greek Restaurant,Halal Restaurant,Hawaiian Restaurant,Hot Dog Joint,Hotpot Restaurant,Hunan Restaurant,Indian Chinese Restaurant,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Latin American Restaurant,Mac & Cheese Joint,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Mongolian Restaurant,New American Restaurant,Noodle House,North Indian Restaurant,Peruvian Restaurant,Pizza Place,Poke Place,Portuguese Restaurant,Ramen Restaurant,Restaurant,Salad Place,Salvadoran Restaurant,Sandwich Place,Seafood Restaurant,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Bellaire,0.000000,0.025316,0.00,0.012658,0.012658,0.000000,0.037975,0.000000,0.000000,0.025316,0.00,0.012658,0.012658,0.012658,0.000000,0.012658,0.000000,0.000000,0.00,0.012658,0.000000,0.00,0.000000,0.00,0.050633,0.000000,0.00,0.000000,0.126582,0.00,0.00,0.025316,0.00,0.025316,0.000000,0.050633,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.025316,0.025316,0.025316,0.00,0.000000,0.088608,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.139241,0.000000,0.00,0.00,0.037975,0.000000,0.012658,0.075949,0.000000,0.00,0.000000,0.000000,0.025316,0.000000,0.012658,0.00,0.025316,0.00,0.000000,0.012658,0.00,0.012658,0.025316,0.000000
1,Downtown / Midtown,0.000000,0.040000,0.00,0.000000,0.040000,0.000000,0.010000,0.020000,0.010000,0.060000,0.00,0.000000,0.000000,0.020000,0.000000,0.010000,0.000000,0.000000,0.01,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.020000,0.01,0.000000,0.000000,0.00,0.00,0.000000,0.02,0.030000,0.010000,0.020000,0.030000,0.000000,0.01,0.000000,0.010000,0.000000,0.00,0.00,0.000000,0.010000,0.000000,0.050000,0.020000,0.000000,0.00,0.010000,0.110000,0.00,0.01,0.00,0.040000,0.000000,0.00,0.000000,0.050000,0.000000,0.01,0.00,0.000000,0.000000,0.000000,0.040000,0.020000,0.00,0.010000,0.030000,0.010000,0.030000,0.020000,0.00,0.050000,0.00,0.000000,0.020000,0.00,0.000000,0.070000,0.010000
2,Garden Oaks / Oak Forest,0.000000,0.037975,0.00,0.000000,0.088608,0.000000,0.012658,0.000000,0.000000,0.037975,0.00,0.000000,0.000000,0.025316,0.000000,0.025316,0.000000,0.000000,0.00,0.000000,0.063291,0.00,0.012658,0.00,0.037975,0.000000,0.00,0.000000,0.101266,0.00,0.00,0.012658,0.00,0.063291,0.000000,0.063291,0.000000,0.000000,0.00,0.012658,0.012658,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.012658,0.000000,0.00,0.012658,0.101266,0.00,0.00,0.00,0.012658,0.000000,0.00,0.000000,0.075949,0.000000,0.00,0.00,0.025316,0.000000,0.000000,0.037975,0.012658,0.00,0.000000,0.000000,0.000000,0.000000,0.012658,0.00,0.012658,0.00,0.012658,0.000000,0.00,0.012658,0.025316,0.025316
3,Highland Village,0.010000,0.080000,0.01,0.000000,0.000000,0.000000,0.040000,0.010000,0.020000,0.060000,0.00,0.000000,0.030000,0.020000,0.000000,0.030000,0.000000,0.010000,0.01,0.000000,0.020000,0.02,0.000000,0.00,0.000000,0.000000,0.01,0.000000,0.020000,0.00,0.00,0.000000,0.01,0.010000,0.040000,0.010000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.070000,0.030000,0.000000,0.00,0.000000,0.050000,0.00,0.00,0.01,0.050000,0.010000,0.00,0.010000,0.

In [19]:
num_top_venues = 5

for hood in houston_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = houston_grouped[houston_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bellaire----
                  venue  freq
0           Pizza Place  0.14
1  Fast Food Restaurant  0.13
2    Mexican Restaurant  0.09
3        Sandwich Place  0.08
4            Donut Shop  0.05


----Downtown / Midtown----
                   venue  freq
0     Mexican Restaurant  0.11
1  Vietnamese Restaurant  0.07
2           Burger Joint  0.06
3     Italian Restaurant  0.05
4            Pizza Place  0.05


----Garden Oaks / Oak Forest----
                  venue  freq
0    Mexican Restaurant  0.10
1  Fast Food Restaurant  0.10
2             BBQ Joint  0.09
3           Pizza Place  0.08
4   Fried Chicken Joint  0.06


----Highland Village----
                 venue  freq
0  American Restaurant  0.08
1   Italian Restaurant  0.07
2         Burger Joint  0.06
3   Mexican Restaurant  0.05
4       Sandwich Place  0.05


----Medical Center----
                 venue  freq
0          Pizza Place  0.10
1  American Restaurant  0.09
2       Sandwich Place  0.08
3           Restaurant  0.06
4 

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = houston_grouped['Neighborhood']

for ind in np.arange(houston_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(houston_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bellaire,Pizza Place,Fast Food Restaurant,Mexican Restaurant,Sandwich Place,Donut Shop,Fried Chicken Joint,Bakery,Restaurant,Japanese Restaurant,Latin American Restaurant
1,Downtown / Midtown,Mexican Restaurant,Vietnamese Restaurant,Burger Joint,Pizza Place,Taco Place,Italian Restaurant,New American Restaurant,BBQ Joint,American Restaurant,Sandwich Place
2,Garden Oaks / Oak Forest,Mexican Restaurant,Fast Food Restaurant,BBQ Joint,Pizza Place,Fried Chicken Joint,Deli / Bodega,Food Truck,Sandwich Place,American Restaurant,Donut Shop
3,Highland Village,American Restaurant,Italian Restaurant,Burger Joint,New American Restaurant,Steakhouse,Sandwich Place,Mexican Restaurant,Sushi Restaurant,French Restaurant,Bakery
4,Medical Center,Pizza Place,American Restaurant,Sandwich Place,Restaurant,Food Truck,Café,Fast Food Restaurant,Burger Joint,Italian Restaurant,Deli / Bodega
5,Memorial,Food,Mexican Restaurant,American Restaurant,Sandwich Place,African Restaurant,Café,Fast Food Restaurant,Italian Restaurant,Japanese Restaurant,Diner
6,Memorial Park / Rice Military,Italian Restaurant,Mexican Restaurant,Taco Place,Sandwich Place,Sushi Restaurant,Pizza Place,American Restaurant,Hawaiian Restaurant,Fried Chicken Joint,Steakhouse
7,Montrose,Italian Restaurant,Mexican Restaurant,Pizza Place,Vietnamese Restaurant,Mediterranean Restaurant,American Restaurant,Japanese Restaurant,Sandwich Place,BBQ Joint,Cajun / Creole Restaurant
8,Museum District,Food Truck,American Restaurant,Café,Mexican Restaurant,Breakfast Spot,Southern / Soul Food Restaurant,Sandwich Place,Food,Vietnamese Restaurant,Pizza Place
9,River Oaks,New American Restaurant,Café,Mexican Restaurant,French Restaurant,Italian Restaurant,Sandwich Place,Sushi Restaurant,Japanese Restaurant,Seafood Restaurant,Steakhouse


In [31]:
# From looking at results above. We know that we do not want to Place our restuarant in 0-Bellaire, 1-Midtown, 2 - Garden Oak, 4- Medical Center, 7- Montrose, 10- Shepherd Kirby as these
#have pizza restaurants in their top 5. 
#The best location are those with NO pizza restuarants in the top 10: Highland Village, Memorial, River Oaks. 
filtered=houston_venues.loc[houston_venues['Neighborhood']=='Memorial',:]


In [35]:
#Map of Memorial Neighborhoods in Houston, TX

address = 'Houston, TX'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_memorial = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(filtered['Venue Latitude'], filtered['Venue Longitude'], filtered['Venue Category']):
    if label=="Pizza Place":
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#cc3131',
        fill_opacity=0.7,
        parse_html=False).add_to(map_houston)  
    else:
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_houston) 
        
map_memorial

In [28]:
# Map of River Oaks Locations
filtered=houston_venues.loc[houston_venues['Neighborhood']=='River Oaks',:]
map_riveroaks = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(filtered['Venue Latitude'], filtered['Venue Longitude'], filtered['Venue Category']):
    if label=="Pizza Place":
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#cc3131',
        fill_opacity=0.7,
        parse_html=False).add_to(map_houston)  
    else:
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_houston) 
        
map_riveroaks

In [33]:
# Map of Memorial Locations
filtered=houston_venues.loc[houston_venues['Neighborhood']=='Memorial',:]
map_riveroaks = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(filtered['Venue Latitude'], filtered['Venue Longitude'], filtered['Venue Category']):
    if label=="Pizza Place":
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#cc3131',
        fill_opacity=0.7,
        parse_html=False).add_to(map_houston)  
    else:
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_houston) 
        
map_riveroaks